In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import astropy.units as u
import astropy.constants as uc

from ipywidgets import interact
import ipywidgets as widgets

plt.style.use("nonos.default")

from kintomo.ktm import Sculpture, Shape, Velocity, Grid, Geometry, projection_deposition_visible

In [ ]:
num_points = int(1e7)
cube = Sculpture.cube(num_points)
unit_l = 100.0*u.au
unit_m = 1.0*u.M_sun
unit_v = np.sqrt(uc.G*unit_m/unit_l).to(u.km/u.s)

sphere_primary = (cube.x**2 + cube.y**2 + cube.z**2 < 0.5**2)
sculpture_s1 = cube.carve(
        shape=Shape(sphere_primary)
    ).to(unit=unit_l)
r, theta, phi = sculpture_s1.spherical_coordinates()
velocity_s1 = Velocity(
        geometry=Geometry("spherical"),
        v1=np.zeros_like(r/unit_l),
        v2=(r/unit_l),
        v3=np.zeros_like(r/unit_l),
    ).to_cartesian(phi=phi, theta=theta).to(unit=unit_v)

sphere_secondary = ((cube.x-0.8)**2 + cube.y**2 + cube.z**2 < 0.2**2)
sculpture_s2 = cube.carve(
        shape=Shape(sphere_secondary)
    ).to(unit=unit_l)
r, theta, phi = sculpture_s2.spherical_coordinates(x_offset=0.8*unit_l)
velocity_s2 = Velocity(
        geometry=Geometry("spherical"),
        v1=np.zeros_like(r/unit_l),
        v2=(r/unit_l),
        v3=np.zeros_like(r/unit_l),
    ).to_cartesian(phi=phi, theta=theta).to(unit=unit_v)

(sculpture, velocity) = (sculpture_s1+sculpture_s2, velocity_s1+velocity_s2)

# nx, ny, nz = (65, 65, 33)
nx, ny, nz = (255, 255, 33)
grid = Grid.encompass(
    sculpture=sculpture, 
    dimension=(nx, ny, nz),
)

In [15]:
vshift_extent = velocity.shift(get="mean")
dv = 0.02*vshift_extent

def plot_vmap(inclination, vshift):
    v_visible = projection_deposition_visible(
        grid=grid,
        sculpture=sculpture,
        velocity=velocity,
        angle=inclination,
        method="ngp",
    )

    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.pcolormesh(
        "x",
        "y",
        v_visible.T,
        data=grid.cell_edges,
        vmin=-vshift_extent, 
        vmax=vshift_extent,
        cmap="RdBu_r",
    )
    contours = ax.contour(
        "x", 
        "y", 
        v_visible.T, 
        data=grid.cell_centers, 
        levels=np.linspace(vshift-dv,vshift+dv+0.003,10), 
        colors="k", 
        linestyles="solid"
    )
    ax.set(
        aspect="equal", 
        ylim=((3/5)*grid.yedge.min(), (3/5)*grid.yedge.max()), 
        xlabel=f"x [{sculpture.get_unit}]", 
        ylabel=f"y [{sculpture.get_unit}]"
    )

    from mpl_toolkits.axes_grid1 import make_axes_locatable

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(
        im, cax=cax, orientation="vertical"
    )
    cbar.set_label(rf"$v_{{\rm proj}}$ [{velocity.get_unit}]")
    plt.show()

inclination_widget = widgets.FloatSlider(value = 40,
                          min = -90, 
                          max = 90,
                          step = 1.0,
                          description = "inclination",
                          continuous_update = False)

vshift_widget = widgets.FloatSlider(value = 0,
                          min = -vshift_extent, 
                          max = vshift_extent,
                          step = 0.1,
                          description = "vshift",
                          continuous_update = False)

interact(plot_vmap, inclination=inclination_widget, vshift=vshift_widget)

interactive(children=(FloatSlider(value=40.0, continuous_update=False, description='inclination', max=90.0, mi…

<function __main__.plot_vmap(inclination, vshift)>